# Import Libraries

In [1]:
import json
import requests
import pandas as pd
import re
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
from IPython.core.display import HTML
import time
import numpy as np

# Pokemon Go API

## Pokemon names

In [2]:
url = "https://pokemon-go1.p.rapidapi.com/pokemon_names.json"

headers = {
    'x-rapidapi-host': "pokemon-go1.p.rapidapi.com",
    'x-rapidapi-key': "9da2ca79a9mshd70830bd5334fc5p1af8c6jsn2413a2984da8"
    }

response = requests.request("GET", url, headers=headers)
result   = response.json()

### List of names of the different pokemons available in the Pokemon universe

In [3]:
l_pokemon_name=[poke for poke in [poke_info for poke_id, poke_info in result.items()]]

df_pokemon_name = pd.DataFrame()
df_pokemon_name = json_normalize(l_pokemon_name)
df_pokemon_name.columns = [f'pokemon_{col}' for col in df_pokemon_name.columns]

In [4]:
df_pokemon_name.head()

,pokemon_id,pokemon_name
0,1,Bulbasaur
1,2,Ivysaur
2,3,Venusaur
3,4,Charmander
4,5,Charmeleon


## Released Pokemon

In [5]:
url = "https://pokemon-go1.p.rapidapi.com/released_pokemon.json"
response = requests.request("GET", url, headers=headers)
result=response.json()

### List of the pokemon released in the Pokemon Go game: It's a different world than in the console games, with different rules so it's good to narrow the data

In [6]:
l_pokemon_rel=[poke for poke in [poke_info for poke_id, poke_info in result.items()]]

df_pokemon_rel = pd.DataFrame()
df_pokemon_rel = json_normalize(l_pokemon_rel)
df_pokemon_rel.columns = [f'pokemon_{col}' for col in df_pokemon_rel.columns]

In [7]:
df_pokemon_rel.head()

,pokemon_id,pokemon_name
0,1,Bulbasaur
1,10,Caterpie
2,100,Voltorb
3,101,Electrode
4,102,Exeggcute


In [8]:
print(df_pokemon_name.shape)
print(df_pokemon_rel.shape)

(809, 2)
(585, 2)


## Pokemon types and forms

In [9]:
url = "https://pokemon-go1.p.rapidapi.com/pokemon_types.json"
response = requests.request("GET", url, headers=headers)
result=response.json()

### A pokemon can have more than one type (usually from one to two), ex. Grass, water or fire. Also one pokemon can have more than one type in Pokemon GO, so the primary key to be the pokemon id and the form of the pokemon. We find the data needs to be cleaned up a little bit to relex the game standard: Mewtwo Armored pokemon it's set to Mewtwo A. There's a lot of nulls that are set to be Normal pokemon form.We hot encode this variable to be more readable. 

In [10]:
df_pokemon_type = pd.DataFrame()
df_pokemon_type = json_normalize(result)
df_type = df_pokemon_type.type.apply(pd.Series)
df_type.columns=['poke_w1','poke_w2']
df_pokemon_type.loc[df_pokemon_type.form.isnull(), 'form'] = 'Normal'
df_pokemon_type.loc[df_pokemon_type.form== 'A', 'form'] = 'Armored'
df_pokemon_type = pd.concat([df_pokemon_type[:], df_type[:]], axis=1)
df_dummies_raw = pd.get_dummies(df_pokemon_type[['poke_w1','poke_w2']], dummy_na=False)
l_types = pd.concat([df_type['poke_w1'],df_type['poke_w2']]).unique()
l_types = l_types[~pd.isnull(l_types)]

for types in l_types:
    df_dummies_raw[types] = (df_dummies_raw[f'poke_w1_{types}']==1) | (df_dummies_raw[f'poke_w2_{types}']==1)
    df_dummies_raw[types] = df_dummies_raw[types].astype('int')

df_dummies = df_dummies_raw[l_types]
df_pokemon_type = pd.concat([df_pokemon_type[:], df_dummies[:]], axis=1)
df_pokemon_type = df_pokemon_type.drop(['type' , 'poke_w1', 'poke_w2'] , axis='columns')
df_pokemon_type.head()

,form,pokemon_id,pokemon_name,Grass,Fire,Water,Bug,Normal,Dark,Poison,...,Ice,Ground,Fairy,Fighting,Psychic,Rock,Ghost,Dragon,Steel,Flying
0,Fall_2019,1,Bulbasaur,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,Normal,1,Bulbasaur,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Purified,1,Bulbasaur,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Shadow,1,Bulbasaur,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,Normal,2,Ivysaur,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


## Pokemon stats

In [11]:
url = "https://pokemon-go1.p.rapidapi.com/pokemon_stats.json"
response = requests.request("GET", url, headers=headers)
result=response.json()

### Each pokemon have different features: HP (vital points), attack (damage that can be inflicted) and deffense in the case of attack. We get those values

In [12]:
df_pokemon_stats = pd.DataFrame()
df_pokemon_stats = json_normalize(result)
df_pokemon_stats = df_pokemon_stats.rename(columns={'base_stamina':'base_hp'})
df_pokemon_stats.loc[df_pokemon_stats.form.isnull(), 'form'] = 'Normal'
df_pokemon_stats.loc[df_pokemon_stats.form== 'A', 'form'] = 'Armored'
df_pokemon_stats.head()

,base_attack,base_defense,base_hp,form,pokemon_id,pokemon_name
0,118,111,128,Fall_2019,1,Bulbasaur
1,118,111,128,Normal,1,Bulbasaur
2,118,111,128,Purified,1,Bulbasaur
3,118,111,128,Shadow,1,Bulbasaur
4,151,143,155,Normal,2,Ivysaur


## Pokemon max capacity points

In [13]:
url = "https://pokemon-go1.p.rapidapi.com/pokemon_max_cp.json"
response = requests.request("GET", url, headers=headers)
result=response.json()

### A pokemon can level up until a max capacity (usually level 40) depending on the actual hp, attack and deffense. We store the max capacity of a Pokemon, and clean the data

In [14]:
df_pokemons_max_cp = pd.DataFrame()
df_pokemons_max_cp = json_normalize(result)
df_pokemons_max_cp.loc[df_pokemons_max_cp.form.isnull(), 'form'] = 'Normal'
df_pokemons_max_cp.loc[df_pokemons_max_cp.form== 'A', 'form'] = 'Armored'
df_pokemons_max_cp.head()

,form,max_cp,pokemon_id,pokemon_name
0,Fall_2019,1115,1,Bulbasaur
1,Normal,1115,1,Bulbasaur
2,Purified,1115,1,Bulbasaur
3,Shadow,1115,1,Bulbasaur
4,Normal,1699,2,Ivysaur


## Put together all the requests in one dataset. We clean the data a little bit to match exactly the Pokemon Go universe. Also we create a boolean variable to set if the pokemon it's available in the game

In [15]:
df_pokemon_api = pd.merge(df_pokemon_stats, df_pokemon_type, how='left', on=['pokemon_id', 'pokemon_name', 'form'], suffixes=('', '_rel'))
df_pokemon_api = pd.merge(df_pokemon_api, df_pokemons_max_cp, how='left', on=['pokemon_id', 'pokemon_name', 'form'], suffixes=('', '_rel'))
df_pokemon_api = pd.merge(df_pokemon_api, df_pokemon_rel, how='left', on=['pokemon_id'], suffixes=('', '_rel'))
df_pokemon_api['bol_release'] = ~(df_pokemon_api.pokemon_name_rel.isnull())
df_pokemon_api['bol_release'] = df_pokemon_api.bol_release.astype('int')
df_pokemon_api.loc[((df_pokemon_api.pokemon_id == 263) & (df_pokemon_api.form == 'Galarian')) |
               ((df_pokemon_api.pokemon_id == 264) & (df_pokemon_api.form == 'Galarian')) |
               ((df_pokemon_api.pokemon_id == 555) & (df_pokemon_api.form == 'Zen')) |
               ((df_pokemon_api.pokemon_id == 641) & (df_pokemon_api.form == 'Therian')),'bol_release']=0

columns = ['pokemon_id','pokemon_name', 'form', 'bol_release', 'max_cp', 'base_hp', 'base_attack', 'base_defense', 
        'Grass', 'Fire', 'Water', 'Bug', 'Normal', 'Dark',
       'Poison', 'Electric', 'Ice', 'Ground', 'Fairy', 'Fighting', 'Psychic',
       'Rock', 'Ghost', 'Dragon', 'Steel', 'Flying']
df_pokemon_api= df_pokemon_api[columns]
df_pokemon_api.head()

,pokemon_id,pokemon_name,form,bol_release,max_cp,base_hp,base_attack,base_defense,Grass,Fire,...,Ice,Ground,Fairy,Fighting,Psychic,Rock,Ghost,Dragon,Steel,Flying
0,1,Bulbasaur,Fall_2019,1,1115,128,118,111,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,Bulbasaur,Normal,1,1115,128,118,111,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,Bulbasaur,Purified,1,1115,128,118,111,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,Bulbasaur,Shadow,1,1115,128,118,111,1,0,...,0,0,0,0,0,0,0,0,0,0
4,2,Ivysaur,Normal,1,1699,155,151,143,1,0,...,0,0,0,0,0,0,0,0,0,0


# Pokemon go Web

In [16]:
url = 'https://pokemondb.net/go/pokedex'
html=requests.get(url).content
soup = BeautifulSoup(html,'lxml')
print(soup)

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<title>Pokémon GO list of Pokémon | Pokémon Database</title>
<link href="https://fonts.gstatic.com" rel="preconnect"/>
<link href="https://img.pokemondb.net" rel="preconnect"/>
<link href="/static/css/pokemondb-7c22d512a3.css" rel="stylesheet"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="Complete Pokédex for all Pokémon available in Pokémon GO." name="description" property="og:description"/>
<link href="https://pokemondb.net/go/pokedex" rel="canonical"/>
<meta content="https://pokemondb.net/go/pokedex" property="og:url"/>
<meta content="Pokémon GO list of Pokémon" property="og:title"/>
<meta content="summary" name="twitter:card"/>
<link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/apple-touch-icon-precomposed.png" rel="apple-touch-icon-precomposed"/>
<link href="https://pokemondb.net/news/feed" rel="alternate" title="The Pokémon Database newsfeed" type

### We scrap the data from https://pokemondb.net/go/pokedex, to extract the other features of the pokemons: % catch a pokemon, % of a pokemon scape, it's combat moves and the link of the picture

In [17]:
soup_h_pokemon = soup.find(id="pokedex").thead.tr
header_pokemon = ['pokemon_id', 'pokemon_name', 'type', 'attack', 'defense', 'base_hp', 'pct_catch', 'pct_flee',
 'fast_moves', 'charge_moves', 'img_link']
soup_b_pokemon = soup.find(id="pokedex").tbody
body_t_pokemon = [pokemon for pokemon in soup_b_pokemon.find_all("tr")]

list_pokemon=[]
for pokemon in body_t_pokemon:
    image_pokemon = pokemon.find_all('span', class_='img-fixed icon-pkmn')[0].get_attribute_list('data-src')
    data_pokemon  = [pok.text for pok in pokemon.find_all("td")] + image_pokemon
    list_pokemon.append(data_pokemon)
    
df_pokemon_web = pd.DataFrame(list_pokemon, columns= header_pokemon)
df_pokemon_web['pokemon_id'] = df_pokemon_web.pokemon_id.astype(int)

### The data of the pokemons' form it's a little bit messy and doesn't match with the data from the API, so we clean it to match the dataframes

In [18]:
def type_form(name):
    types=['Fall_2019', 'Normal', 'Purified', 'Shadow', 'Alolan',
       'Galarian', 'Armored', 'Rainy', 'Snowy', 'Sunny', 'Attack',
       'Defense', 'Speed', 'Plant', 'Sandy', 'Trash', 'Overcast',
       'East', 'West', 'Fan', 'Frost', 'Heat', 'Mow', 'Wash',
       'Altered', 'Origin', 'Land', 'Sky', 'Bug', 'Dark', 'Dragon',
       'Electric', 'Fairy', 'Fighting', 'Fire', 'Flying', 'Ghost',
       'Grass', 'Ground', 'Ice', 'Poison', 'Psychic', 'Rock', 'Steel',
       'Water', 'Blue', 'Red', 'Standard', 'Zen',
       'Autumn', 'Spring', 'Summer', 'Winter', 'Incarnate', 'Therian',
       'Black', 'White', 'Ordinary', 'Resolute', 'Aria', 'Pirouette',
       'Burn', 'Chill', 'Douse', 'Shock','Sunshine']
    types_lower= [t for t in types if (t.lower() in name.lower().split(' '))]
    if len(types_lower)>0:
        if types_lower[0]=='Sunshine':
            form = 'Sunny'
        elif types_lower[0]=='Alolan':
            form = 'Alola'
        elif types_lower[0]=='East' or types_lower[0]=='West':
            form = f'{types_lower[0]}_sea'
        elif types_lower[0]=='Blue' or types_lower[0]=='Red':
            form = f'{types_lower[0]}_striped'
        else:
            form = types_lower[0]
    else:
        form = 'Normal'
    return(form)

df_pokemon_web['form'] = df_pokemon_web.pokemon_name.apply(lambda x:type_form(x.replace('-',' ')))
df_pokemon_web.head()

,pokemon_id,pokemon_name,type,attack,defense,base_hp,pct_catch,pct_flee,fast_moves,charge_moves,img_link,form
0,1,Bulbasaur,Grass Poison,128,118,111,20%,10%,Vine WhipTackle,Seed BombSludge BombPower WhipFrustration (Sha...,https://img.pokemondb.net/sprites/sword-shield...,Normal
1,2,Ivysaur,Grass Poison,155,151,143,10%,7%,Vine WhipRazor Leaf,Sludge BombSolar BeamPower WhipFrustration (Sh...,https://img.pokemondb.net/sprites/sword-shield...,Normal
2,3,Venusaur,Grass Poison,190,198,189,5%,5%,Vine WhipRazor Leaf,Petal BlizzardSludge BombSolar BeamFrenzy Plan...,https://img.pokemondb.net/sprites/sword-shield...,Normal
3,4,Charmander,Fire,118,116,93,20%,10%,EmberScratch,FlamethrowerFlame ChargeFlame BurstFrustration...,https://img.pokemondb.net/sprites/sword-shield...,Normal
4,5,Charmeleon,Fire,151,158,126,10%,7%,EmberFire Fang,FlamethrowerFlame BurstFire PunchFrustration (...,https://img.pokemondb.net/sprites/sword-shield...,Normal


# We merge the Web and API dataframes

In [19]:
df_pokemon = pd.merge(df_pokemon_api, df_pokemon_web, how='inner', on=['pokemon_id', 'form'], suffixes=('', '_rel'))
df_pokemon.head()

,pokemon_id,pokemon_name,form,bol_release,max_cp,base_hp,base_attack,base_defense,Grass,Fire,...,pokemon_name_rel,type,attack,defense,base_hp_rel,pct_catch,pct_flee,fast_moves,charge_moves,img_link
0,1,Bulbasaur,Normal,1,1115,128,118,111,1,0,...,Bulbasaur,Grass Poison,128,118,111,20%,10%,Vine WhipTackle,Seed BombSludge BombPower WhipFrustration (Sha...,https://img.pokemondb.net/sprites/sword-shield...
1,2,Ivysaur,Normal,1,1699,155,151,143,1,0,...,Ivysaur,Grass Poison,155,151,143,10%,7%,Vine WhipRazor Leaf,Sludge BombSolar BeamPower WhipFrustration (Sh...,https://img.pokemondb.net/sprites/sword-shield...
2,3,Venusaur,Normal,1,2720,190,198,189,1,0,...,Venusaur,Grass Poison,190,198,189,5%,5%,Vine WhipRazor Leaf,Petal BlizzardSludge BombSolar BeamFrenzy Plan...,https://img.pokemondb.net/sprites/sword-shield...
3,4,Charmander,Normal,1,980,118,116,93,0,1,...,Charmander,Fire,118,116,93,20%,10%,EmberScratch,FlamethrowerFlame ChargeFlame BurstFrustration...,https://img.pokemondb.net/sprites/sword-shield...
4,5,Charmeleon,Normal,1,1653,151,158,126,0,1,...,Charmeleon,Fire,151,158,126,10%,7%,EmberFire Fang,FlamethrowerFlame BurstFire PunchFrustration (...,https://img.pokemondb.net/sprites/sword-shield...


In [20]:
columns = ['pokemon_id', 'pokemon_name', 'form', 'bol_release', 'max_cp',
       'base_hp', 'base_attack', 'base_defense', 'Grass', 'Fire', 'Water',
       'Bug', 'Normal', 'Dark', 'Poison', 'Electric', 'Ice', 'Ground', 'Fairy',
       'Fighting', 'Psychic', 'Rock', 'Ghost', 'Dragon', 'Steel', 'Flying',
       'pct_catch', 'pct_flee', 'fast_moves', 'charge_moves', 'img_link']
df_pokemon = df_pokemon[columns]

In [21]:
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

HTML(df_pokemon.to_html(escape=False ,formatters=dict(img_link=path_to_image_html)))

,pokemon_id,pokemon_name,form,bol_release,max_cp,base_hp,base_attack,base_defense,Grass,Fire,Water,Bug,Normal,Dark,Poison,Electric,Ice,Ground,Fairy,Fighting,Psychic,Rock,Ghost,Dragon,Steel,Flying,pct_catch,pct_flee,fast_moves,charge_moves,img_link
0,1,Bulbasaur,Normal,1,1115,128,118,111,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,20%,10%,Vine WhipTackle,Seed BombSludge BombPower WhipFrustration (Sha...,"<img src=""https://img.pokemondb.net/sprites/sw..."
1,2,Ivysaur,Normal,1,1699,155,151,143,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,10%,7%,Vine WhipRazor Leaf,Sludge BombSolar BeamPower WhipFrustration (Sh...,"<img src=""https://img.pokemondb.net/sprites/sw..."
2,3,Venusaur,Normal,1,2720,190,198,189,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5%,5%,Vine WhipRazor Leaf,Petal BlizzardSludge BombSolar BeamFrenzy Plan...,"<img src=""https://img.pokemondb.net/sprites/sw..."
3,4,Charmander,Normal,1,980,118,116,93,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,20%,10%,EmberScratch,FlamethrowerFlame ChargeFlame BurstFrustration...,"<img src=""https://img.pokemondb.net/sprites/sw..."
4,5,Charmeleon,Normal,1,1653,151,158,126,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10%,7%,EmberFire Fang,FlamethrowerFlame BurstFire PunchFrustration (...,"<img src=""https://img.pokemondb.net/sprites/sw..."
5,6,Charizard,Normal,1,2889,186,223,173,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,5%,5%,Air SlashFire Spin,Dragon ClawFire BlastOverheatBlast Burn (Commu...,"<img src=""https://img.pokemondb.net/sprites/sw..."
6,7,Squirtle,Normal,1,946,127,94,121,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,20%,10%,TackleBubble,Aqua JetAqua TailWater PulseFrustration (Shado...,"<img src=""https://img.pokemondb.net/sprites/sw..."
7,8,Wartortle,Normal,1,1488,153,126,155,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10%,7%,BiteWater Gun,Ice BeamAqua JetHydro PumpFrustration (Shadow)...,"<img src=""https://img.pokemondb.net/sprites/sw..."
8,9,Blastoise,Normal,1,2466,188,171,207,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5%,5%,BiteWater Gun,Flash CannonIce BeamHydro PumpHydro Cannon (Co...,"<img src=""https://img.pokemondb.net/sprites/sw..."
9,10,Caterpie,Normal,1,437,128,55,55,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,50%,20%,Bug BiteTackle,Struggle,"<img src=""https://img.pokemondb.net/sprites/sw..."


# Exportamos la información a csv

In [22]:
df_pokemon_api.to_csv('./output/pokemon_go_api.csv', index=False)
df_pokemon_web.to_csv('./output/pokemon_go_web.csv', index=False)
df_pokemon.to_csv('./output/pokemon_go_stats.csv', index=False)